# 🧠 02_entrenamiento_modelo.ipynb

Este notebook entrena un modelo de aprendizaje profundo (Deep Learning) para reconocer letras del lenguaje de señas usando los *landmarks* generados por MediaPipe.

## 🔄 Etapas del notebook:
1. Importar librerías
2. Cargar los datos limpios
3. Separar características y etiquetas
4. Codificar las etiquetas
5. Dividir el dataset en entrenamiento y prueba
6. Normalizar los datos
7. Crear el modelo neuronal
8. Compilar el modelo
9. Entrenar el modelo
10. Visualizar resultados


## 🧩 1️⃣ Importar librerías

Importamos todas las librerías necesarias para el manejo de datos, construcción del modelo y visualización de resultados.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt

## 🧹 2️⃣ Cargar los datos limpios

Cargamos el dataset que fue previamente limpiado en el notebook `01_limpieza_datos.ipynb`. Este archivo contiene los landmarks de las manos normalizados y una columna con la etiqueta (letra).

In [ ]:
data = pd.read_csv("/content/dataset_cleaned.csv")  # Cambia la ruta si está en otra ubicación
data.head()

## ⚙️ 3️⃣ Separar características (X) y etiquetas (y)

Separamos los datos en:
- **X:** las coordenadas de los 21 puntos de la mano (x, y, z)
- **y:** la letra correspondiente a cada muestra

In [ ]:
X = data.drop('label', axis=1)
y = data['label']

## 🧮 4️⃣ Codificar las etiquetas

Convertimos las letras (A, B, C...) en números para que el modelo pueda entenderlas, y luego usamos *one-hot encoding* para representar cada letra como un vector binario.

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

print("Etiquetas codificadas:", np.unique(y_encoded))
print("Forma de y_categorical:", y_categorical.shape)

## 🧩 5️⃣ Dividir el dataset en entrenamiento y prueba

Reservamos un 80% de los datos para entrenamiento y un 20% para validación. Esto nos permite evaluar el desempeño del modelo con datos que nunca ha visto.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42
)

print("Tamaño de entrenamiento:", X_train.shape)
print("Tamaño de prueba:", X_test.shape)

## ⚖️ 6️⃣ Normalizar los datos

Los valores de los landmarks pueden variar mucho dependiendo de la imagen, por eso los normalizamos usando `StandardScaler` para que todos estén en una escala similar. Esto mejora el aprendizaje del modelo.

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 🧠 7️⃣ Crear el modelo neuronal

Diseñamos una red neuronal totalmente conectada (Feedforward Neural Network):
- Capa 1: 128 neuronas con activación ReLU.
- Dropout: 30% (reduce sobreajuste).
- Capa 2: 64 neuronas con activación ReLU.
- Dropout: 30%.
- Capa de salida: tantas neuronas como letras, con activación *softmax*.

In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')
])

model.summary()

## ⚙️ 8️⃣ Compilar el modelo

Seleccionamos el optimizador, la función de pérdida y la métrica de evaluación:
- **Adam:** un optimizador rápido y eficiente.
- **Categorical Crossentropy:** adecuada para clasificación multiclase.
- **Accuracy:** mide el porcentaje de predicciones correctas.

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## 🚀 9️⃣ Entrenar el modelo

Entrenamos la red neuronal durante 50 épocas con un tamaño de lote (*batch size*) de 32. Se mostrará cómo evoluciona la pérdida y la precisión en cada iteración.

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)

## 📊 🔟 Visualizar resultados

Graficamos la precisión (*accuracy*) de entrenamiento y validación para ver si el modelo está aprendiendo correctamente o si se está sobreajustando.

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión del modelo')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

## 💾 1️⃣1️⃣ Guardar el modelo entrenado

Guardamos el modelo y el codificador de etiquetas para poder usarlos más adelante en la predicción en tiempo real.

In [ ]:
model.save("/content/modelo_senas.h5")
import joblib
joblib.dump(le, "/content/label_encoder.pkl")
print("Modelo y codificador guardados correctamente ✅")

## ✅ Conclusión

En este notebook entrenamos un modelo neuronal para reconocer letras del lenguaje de señas basado en los landmarks 3D de MediaPipe. 

El siguiente paso será crear un notebook para la **predicción en tiempo real**, usando la cámara y el modelo `modelo_senas.h5`.